In [ ]:
import socket
import os
import sys
import struct

try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # IP地址留空默认是本机IP地址
        s.bind(('192.168.50.1', 8003))
        s.listen(7)
except socket.error as msg:
        print(msg)
        sys.exit(1)
        
print("连接开启，等待传txt...")


In [ ]:
def run_stop_get():
    pwm_ENA.start(0)
    pwm_ENB.start(0)
    
def run_stop_put():
    pwm_ENA.stop()
    pwm_ENB.stop()
    

In [ ]:
# @Time   : 2022/9/11 11:15
# @Author :lgl
# @e-mail :GuanlinLi_BIT@163.com
def read_result(file_route):
    global y1, x2, y2, x1
    file = open(file_route, 'r')
    line1 = file.readline()
    label = file.readline()
    line3 = file.readline()
    line4 = file.readline()
    score = file.readline()
    line5 = file.readline()
    line6 = file.readline()
    coordinate = file.readline()

    label = (label.split("[")[1]).split(".")[0]
    score = (score.split("[")[1]).split("]")[0]
    coordinate = coordinate.split(" ")
    num = 0
    for i in coordinate:
        if len(i) != 1 and len(i) != 0 and num == 0:
            x1 = i
            num += 1
            continue
        if len(i) != 1 and len(i) != 0 and num == 1:
            y1 = i
            num += 1
            continue
        if len(i) != 1 and len(i) != 0 and num == 2:
            x2 = i
            num += 1
            continue
        if len(i) != 1 and len(i) != 0 and num == 3:
            y2 = i
            num += 1
            continue
    x1 = x1.split("[")
    x1 = x1[len(x1)-1]
    y2 = y2.split("]")[0]
    return label, score, x1, y1, x2, y2

#read_result("/home/pi/Desktop/gluoncv/result.txt")
#print(x1,y1,x2,y2)


In [ ]:
import socket
import os
import sys
import struct
 
def upload():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        #192.168.199.1和8088分别为服务端（pc）的IP地址和网络端口
        s.connect(('192.168.50.107', 8086))
    except socket.error as msg:
        print(msg)
        print(sys.exit(1))
    
    video = cv2.VideoCapture("http://192.168.50.1:8080/?action=stream")
    cv2.namedWindow("img",cv2.WINDOW_NORMAL)
    ret,frame = video.read()
    time.sleep(2)
    if(frame is None):
        print("2")
    cv2.imwrite("/home/pi/Desktop/gluoncv/image.png", frame)
    
    while True:
        #filepath是要被发送图片的路径
        filepath = '/home/pi/Desktop/gluoncv/image.png'
        fhead = struct.pack(b'128sl', bytes(os.path.basename(filepath), encoding='utf-8'), os.stat(filepath).st_size)
        s.send(fhead)
        print('client filepath: {0}'.format(filepath))
 
        fp = open(filepath, 'rb')
        while 1:
            data = fp.read(1024)
            if not data:
                print('{0} 发送成功...'.format(filepath))
                break
            s.send(data)
        s.close()
        break


In [ ]:
from random import random


def move(s, x1, y1, x2, y2, t):
    s = float(s)
    x1 = float(x1)
    y1 = float(y1)
    x2 = float(x2)
    y2 = float(y2)
    t = float(t)
    if s >= -1 and s<0.01:  # 得分过低，找不到目标，需要随机移动，直到找到东西
        right_move(t)
    elif s== -2:
        run_stop_put()
    else:
        if y2 - y1 > 250:
            if x1 > 640 or x2 > 640 or x1 < 0 or x2 < 0 or y1 > 480 or y2 > 480 or y1 < 0 or y2 < 0:
                # 图像显示不全，需要后退
                back(t / 2)
                print("back")
            else:
                if (x1 + x2) / 2 > 400:  # 目标偏左
                    x=(((x1+x2)/2)-340)
                    t=x/680*t
                    right_move(t)
                    print("close_right_move")
                if (x1 + x2) / 2 < 240:  # 目标偏右
                    x=(340-((x1+x2)/2))
                    t=x/680*t
                    left_move(t)
                    print("far_left_move")
                if 240 <= (x1 + x2) / 2 <= 400:  # 位置合适，不动，接着拍摄即可# 找到了目标，但是距离太远，需要接近目标
                    print("位置合适,ok")
                    run_stop_put()                   
        else:
            if y1 > 480 or y2 > 480 or y1 < 0 or y2 < 0:
                # 图像显示不全,需要后退
                back(t / 2)
                print("back")
            else:
                if (x1 + x2) / 2 > 400 or x1<20 or x2<20:# 目标偏左
                    t=(x2-x1)/680*2*t
                    right_move(t)
                    print("right_move")
                if (x1 + x2) / 2 < 240 or x2>640 or x1>640:  # 目标偏右
                    t=(x2-x1)/680*2*t
                    left_move(t)
                    print("left_move")
                if  240 <= (x1 + x2) / 2 <= 400:# 位置合适找到了目标，但是距离太远，需要接近目标
                    forward(t)
                    print("forward")

In [ ]:
import RPi.GPIO as GPIO         # 引入GPIO模块
import time                     # 引入time模块

IN1 = 20
IN2 = 21
IN3 = 19
IN4 = 26
ENA = 16
ENB = 13
speed=10
speed2=30
#设置GPIO口为BCM编码方式
GPIO.setmode(GPIO.BCM)
# #忽略警告信息
# GPIO.setwarnings(False)

# GPIO.setup(ENA,GPIO.OUT,initial=GPIO.HIGH)
# GPIO.setup(IN1,GPIO.OUT,initial=GPIO.LOW)
# GPIO.setup(IN2,GPIO.OUT,initial=GPIO.LOW)
# GPIO.setup(ENB,GPIO.OUT,initial=GPIO.HIGH)
# GPIO.setup(IN3,GPIO.OUT,initial=GPIO.LOW)
# GPIO.setup(IN4,GPIO.OUT,initial=GPIO.LOW)
# #设置pwm引脚和频率为2000hz
# pwm_ENA = GPIO.PWM(ENA, 2000)
# pwm_ENB = GPIO.PWM(ENB, 2000)

# pwm_ENA.stop()
# pwm_ENB.stop()
 
# pwm_ENA.start(0)
# pwm_ENB.start(0)
    
def forward(t):
    run_stop_get()
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)
    pwm_ENA.ChangeDutyCycle(speed)
    pwm_ENB.ChangeDutyCycle(speed)
    time.sleep(t)
    run_stop_put()


def back(t):
    run_stop_get()
    GPIO.setmode(GPIO.BCM)
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.HIGH)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.HIGH)
    pwm_ENA.ChangeDutyCycle(speed)
    pwm_ENB.ChangeDutyCycle(speed)
    time.sleep(t/2)   
    run_stop_put()
    
def right_move(t):
    run_stop_get()
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.LOW)
    pwm_ENA.ChangeDutyCycle(speed)
    pwm_ENB.ChangeDutyCycle(0)
    time.sleep(t/2)
    run_stop_put()

    
def left_move(t):
    run_stop_get()
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)
    pwm_ENA.ChangeDutyCycle(0)
    pwm_ENB.ChangeDutyCycle(speed)
    time.sleep(t/2)
    run_stop_put()
    
def right_turn(t):
    run_stop_get()
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.HIGH)
    pwm_ENA.ChangeDutyCycle(speed2)
    pwm_ENB.ChangeDutyCycle(speed2)
    time.sleep(t/2)
    run_stop_put()
    
def left_turn(t):
    run_stop_get()
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)
    pwm_ENA.ChangeDutyCycle(speed2)
    pwm_ENB.ChangeDutyCycle(speed2)
    time.sleep(t/2)
    run_stop_put()

In [ ]:
import time
import cv2
# person(人) bicycle(自行车)  car(汽车)  motorbike(摩托车)  aeroplane(飞机)  bus(公共汽车)  train(火车)  truck(卡车)  boat(船) traffic
# light(信号灯)  fire hydrant(消防栓)  stop sign(停车标志)  parking meter(停车计费器)  bench(长凳) bird(鸟)  cat(猫)  dog(狗)  horse(马)
# sheep(羊)  cow(牛)  elephant(大象)  bear(熊)  zebra(斑马)  giraffe(长颈鹿) backpack(背包)  umbrella(雨伞)  handbag(手提包)  tie(领带)
# suitcase(手提箱) frisbee(飞盘)  skis(滑雪板双脚)  snowboard(滑雪板)  sports ball(运动球)  kite(风筝) baseball bat(棒球棒)  baseball
# glove(棒球手套)  skateboard(滑板)  surfboard(冲浪板)  tennis racket(网球拍) bottle(瓶子)  wine glass(高脚杯)  cup(茶杯)  fork(叉子)
# knife(刀) spoon(勺子)  bowl(碗) banana(香蕉)  apple(苹果)  sandwich(三明治)  orange(橘子)  broccoli(西兰花)  carrot(胡萝卜)  hot dog(
# 热狗)  pizza(披萨)  donut(甜甜圈)  cake(蛋糕) chair(椅子)  sofa(沙发)  pottedplant(盆栽植物)  bed(床)  diningtable(餐桌)  toilet(厕所)
# tvmonitor(电视机) laptop(笔记本)  mouse(鼠标)  remote(遥控器)  keyboard(键盘)  cell phone(电话) microwave(微波炉)  oven(烤箱)  toaster(
# 烤面包器)  sink(水槽)  refrigerator(冰箱) book(书)  clock(闹钟)  vase(花瓶)  scissors(剪刀)  teddy bear(泰迪熊)  hair drier(吹风机)
# toothbrush(牙刷)
import socket
import os
import sys
import struct


def gettxt():
    print("连接开启，等待传txt...")
    sock, addr = s.accept()
    deal_data(sock, addr)

def deal_data(sock, addr):
    print("成功连接上 {0}".format(addr))

    while True:
        fileinfo_size = struct.calcsize('128sl')
        buf = sock.recv(fileinfo_size)
        if buf:
            filename, filesize = struct.unpack('128sl', buf)
            fn = filename.decode().strip('\x00')
            # PC端图片保存路径
            new_filename = os.path.join('/home/pi/Desktop/gluoncv', fn)

            recvd_size = 0
            fp = open(new_filename, 'wb')

            while not recvd_size == filesize:
                if filesize - recvd_size > 1024:
                    data = sock.recv(1024)
                    recvd_size += len(data)
                else:
                    data = sock.recv(1024)
                    recvd_size = filesize
                fp.write(data)
            fp.close()
        sock.close()

        break

        
print("获取资源")
#获取小车车轮资源和视频资源
global pwm_ENA
global pwm_ENB
#忽略警告信息
GPIO.setwarnings(False)

GPIO.setup(ENA,GPIO.OUT,initial=GPIO.HIGH)
GPIO.setup(IN1,GPIO.OUT,initial=GPIO.LOW)
GPIO.setup(IN2,GPIO.OUT,initial=GPIO.LOW)
GPIO.setup(ENB,GPIO.OUT,initial=GPIO.HIGH)
GPIO.setup(IN3,GPIO.OUT,initial=GPIO.LOW)
GPIO.setup(IN4,GPIO.OUT,initial=GPIO.LOW)
#设置pwm引脚和频率为2000hz
pwm_ENA = GPIO.PWM(ENA, 2000)
pwm_ENB = GPIO.PWM(ENB, 2000)

pwm_ENA.stop()
pwm_ENB.stop()
 
pwm_ENA.start(0)
pwm_ENB.start(0)


# video = cv2.VideoCapture(1)
# print(video.isOpened())
# fps = video.get(cv2.CAP_PROP_FPS)
# print(fps)
# size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
# print("size:{}".format(size))
while True:
        upload()
        gettxt()
        label, score, x1, y1, x2, y2 = read_result("/home/pi/Desktop/gluoncv/result.txt")
        print("label=",label)
        print("score=",score)
        print(x1,y1,x2,y2)
        move(score, x1,y1,x2,y2,2)
        


In [ ]:

#video.release()
cv2.destroyAllWindows()
pwm_ENA.stop()
pwm_ENB.stop()
GPIO.cleanup() 